# 132: Kubernetes Fundamentals for ML

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** Kubernetes architecture (control plane, worker nodes, pods, services, deployments)
- **Implement** ML model deployments (declarative YAML, replica management, health checks)
- **Build** auto-scaling systems (HPA based on CPU, custom metrics, traffic patterns)
- **Apply** Kubernetes to post-silicon validation (STDF parser service, wafer analysis API, distributed testing)
- **Master** rolling updates (zero-downtime deployments, canary releases, rollback strategies)
- **Deploy** production ML services (load balancing, service discovery, resource management)

## 📚 What is Kubernetes for ML?

**Kubernetes** (K8s) is a container orchestration platform that automates deployment, scaling, and management of containerized applications. For ML, Kubernetes solves critical production challenges:

**The Deployment Challenge:**
```
Without Kubernetes (Manual):
- Data scientist builds Docker image: wafer_yield_model:v2
- DevOps manually deploys to 10 servers
- Load balancer needs manual configuration
- Server crashes → manual restart (downtime: 15 minutes)
- Need to scale 10 → 20 servers → manual provisioning (2 hours)
- Model update → manual rolling update (error-prone, risky)
```

**Kubernetes Solution:**
```yaml
# deployment.yaml (declarative)
apiVersion: apps/v1
kind: Deployment
metadata:
  name: wafer-yield-model
spec:
  replicas: 10  # Desired state
  template:
    spec:
      containers:
      - name: model
        image: wafer_yield_model:v2
        resources:
          limits:
            cpu: "2"
            memory: "4Gi"
---
# service.yaml (stable endpoint)
apiVersion: v1
kind: Service
metadata:
  name: wafer-yield-model
spec:
  selector:
    app: wafer-yield-model
  ports:
  - port: 80
    targetPort: 8080
```

**What Kubernetes Does Automatically:**
- ✅ **Self-healing:** Pod crashes → new pod started automatically (seconds)
- ✅ **Auto-scaling:** CPU >70% → scale 10 → 15 pods (minutes)
- ✅ **Load balancing:** Service distributes traffic across healthy pods
- ✅ **Rolling updates:** Deploy v3 gradually (v2 pods → v3 pods, zero downtime)
- ✅ **Service discovery:** Pods find services by DNS (wafer-yield-model.default.svc.cluster.local)
- ✅ **Resource management:** Ensures 2 CPU, 4Gi memory per pod (prevents resource starvation)

**Why Kubernetes for ML?**
- ✅ **Scalability:** Handle 10 → 1000 requests/sec automatically (horizontal pod autoscaling)
- ✅ **High Availability:** 99.9% uptime (pod distribution across nodes, auto-restart on failure)
- ✅ **Zero-Downtime Deployments:** Rolling updates with health checks (traffic shifts to new version gradually)
- ✅ **Resource Efficiency:** Bin packing (fit multiple models on same node, maximize GPU utilization)
- ✅ **Multi-Tenancy:** Namespaces isolate teams (team-design, team-test, team-validation)
- ✅ **Cloud Agnostic:** Run on AWS (EKS), Azure (AKS), GCP (GKE), on-prem (same manifests)

## 🏭 Post-Silicon Validation Use Cases

**Use Case 1: STDF Parser Service (Auto-Scaling)**
- **Input:** Test equipment streams STDF files (10-100 wafers/hour, variable rate)
- **Deployment:** Kubernetes Deployment with 5 initial replicas
- **Auto-Scaling:** HPA monitors queue depth (>50 files → scale up to 20 replicas)
- **Output:** Parsed data in Parquet format (sub-5 second p95 latency)
- **Value:** $85K/year savings (eliminate manual server management, auto-scale down during off-peak)

**Use Case 2: Wafer Yield Prediction API (High Availability)**
- **Input:** Manufacturing execution system (MES) requests yield predictions (200 req/sec peak)
- **Deployment:** Deployment with 10 replicas across 3 availability zones
- **Service:** LoadBalancer type service (stable IP, SSL termination)
- **Health Checks:** Liveness probe (restart unhealthy pods), readiness probe (remove from load balancer)
- **Output:** Real-time yield predictions (p99 latency <50ms, 99.95% uptime)
- **Value:** $2.1M/year revenue protection (prevent yield loss from undetected issues)

**Use Case 3: Distributed Test Execution (Batch Processing)**
- **Input:** 500 test jobs queued (regression testing on new silicon)
- **Deployment:** Job resource (parallel execution, 20 pods, completion tracking)
- **Resource Limits:** Each pod gets 4 CPU, 8Gi memory (prevent resource contention)
- **Output:** Test results aggregated (success/failure per test case)
- **Value:** 5x faster execution (20 hours → 4 hours, parallel vs sequential)

**Use Case 4: Model A/B Testing (Canary Release)**
- **Input:** New yield model v2.5 (95% accuracy in validation, needs production verification)
- **Deployment:** Canary deployment (5% traffic to v2.5, 95% to v2.4)
- **Monitoring:** Compare error rates, latency, prediction accuracy (real-time metrics)
- **Rollout Strategy:** If metrics stable, gradually increase to 100% (10%, 25%, 50%, 100%)
- **Value:** Zero-risk deployments (auto-rollback if v2.5 degrades metrics >5%)

## 🔄 Kubernetes Workflow for ML

```mermaid
graph LR
    A[Data Scientist] -->|kubectl apply| B[API Server]
    B -->|Store| C[etcd]
    B -->|Schedule| D[Scheduler]
    D -->|Assign| E[Worker Node 1]
    D -->|Assign| F[Worker Node 2]
    E -->|Run| G[Pod: model-v2]
    F -->|Run| H[Pod: model-v2]
    I[Service] -->|Load Balance| G
    I -->|Load Balance| H
    J[HPA] -->|Monitor| G
    J -->|Monitor| H
    J -->|Scale| B
    
    style A fill:#e1f5ff
    style B fill:#fff4e1
    style C fill:#ffe1e1
    style D fill:#e1ffe1
    style G fill:#f0e1ff
    style H fill:#f0e1ff
    style I fill:#ffe1f5
    style J fill:#e1ffff
```

## 📊 Learning Path Context

**Prerequisites:**
- **Notebook 131:** Docker for ML (containerization fundamentals, multi-stage builds, model serving)

**Current Notebook:**
- **Notebook 132:** Kubernetes Fundamentals (architecture, deployments, services, auto-scaling, rolling updates)

**Next Steps:**
- **Notebook 133:** Kubernetes Advanced Patterns (StatefulSets, DaemonSets, Operators, CRDs)
- **Notebook 134:** Service Mesh (Istio, Linkerd for microservices communication)
- **Notebook 135:** GitOps (ArgoCD, Flux for declarative deployments)

---

Let's orchestrate ML models at scale! 🚀

In [ ]:
# Setup and Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import json
import time
from datetime import datetime
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict, Optional
import uuid

# Set random seed for reproducibility
np.random.seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")

print("✅ Environment ready for Kubernetes ML orchestration simulation")

## 2. 🏗️ Kubernetes Architecture and Core Concepts

**Purpose:** Understand Kubernetes cluster components and how they work together to orchestrate containers at scale.

**Key Points:**
- **Control Plane:** Brain of the cluster (API server, scheduler, controller manager, etcd)
- **Worker Nodes:** Run application pods (kubelet, kube-proxy, container runtime)
- **Pods:** Smallest deployable unit (1+ containers, shared network/storage)
- **Deployments:** Declarative pod management (desired state, rolling updates)
- **Services:** Stable networking for pods (load balancing, service discovery)
- **Namespaces:** Virtual clusters for multi-tenancy (dev, staging, production)

**Why It Matters:**
- **Control plane** makes all decisions (schedule pods, detect failures, scale replicas)
- **Worker nodes** execute workloads (kubelet ensures pods running, kube-proxy handles networking)
- **Pods are ephemeral** (IP changes on restart, need Services for stable endpoints)
- **Deployments enable declarative updates** (specify v2.3, Kubernetes handles rollout)

**Post-Silicon Application:**
STDF parser service deployment: 10 pods across 3 worker nodes, Service provides stable endpoint (`stdf-parser.validation.svc.cluster.local:8080`), test equipment sends files to Service, Kubernetes load balances to healthy pods.

In [ ]:
# Kubernetes Architecture Simulation

@dataclass
class Pod:
    """Represents a Kubernetes Pod (smallest deployable unit)."""
    name: str
    node: str
    status: str = "Running"
    ip: str = ""
    containers: List[str] = field(default_factory=list)
    cpu_request: float = 0.5  # CPU cores
    memory_request: int = 512  # MB
    
    def __post_init__(self):
        if not self.ip:
            # Simulate pod IP (10.244.x.y format)
            self.ip = f"10.244.{np.random.randint(0, 255)}.{np.random.randint(1, 255)}"
    
    def restart(self):
        """Simulate pod restart (new IP assigned)."""
        self.ip = f"10.244.{np.random.randint(0, 255)}.{np.random.randint(1, 255)}"
        self.status = "Running"
        print(f"🔄 Pod {self.name} restarted with new IP: {self.ip}")


@dataclass
class Node:
    """Represents a Kubernetes Worker Node."""
    name: str
    cpu_capacity: float = 8.0  # CPU cores
    memory_capacity: int = 16384  # MB
    pods: List[Pod] = field(default_factory=list)
    
    def available_cpu(self) -> float:
        """Calculate available CPU."""
        used = sum(pod.cpu_request for pod in self.pods if pod.status == "Running")
        return self.cpu_capacity - used
    
    def available_memory(self) -> int:
        """Calculate available memory."""
        used = sum(pod.memory_request for pod in self.pods if pod.status == "Running")
        return self.memory_capacity - used
    
    def can_schedule(self, pod: Pod) -> bool:
        """Check if node has resources for pod."""
        return (self.available_cpu() >= pod.cpu_request and 
                self.available_memory() >= pod.memory_request)
    
    def add_pod(self, pod: Pod) -> bool:
        """Schedule pod on node."""
        if self.can_schedule(pod):
            pod.node = self.name
            self.pods.append(pod)
            return True
        return False


class KubernetesCluster:
    """Simulates a Kubernetes cluster with control plane logic."""
    
    def __init__(self, name: str):
        self.name = name
        self.nodes: List[Node] = []
        self.services: Dict[str, 'Service'] = {}
        
    def add_node(self, node: Node):
        """Add worker node to cluster."""
        self.nodes.append(node)
        print(f"➕ Node {node.name} added to cluster (CPU: {node.cpu_capacity}, Memory: {node.memory_capacity}MB)")
    
    def schedule_pod(self, pod: Pod) -> bool:
        """Schedule pod on node with available resources (simplified scheduler)."""
        # Sort nodes by available resources (bin packing)
        available_nodes = [n for n in self.nodes if n.can_schedule(pod)]
        
        if not available_nodes:
            print(f"❌ Cannot schedule pod {pod.name}: insufficient resources")
            return False
        
        # Choose node with most available resources (spread strategy)
        best_node = max(available_nodes, key=lambda n: n.available_cpu())
        best_node.add_pod(pod)
        print(f"✅ Pod {pod.name} scheduled on {best_node.name} (IP: {pod.ip})")
        return True
    
    def create_deployment(self, name: str, image: str, replicas: int, 
                         cpu_request: float, memory_request: int) -> List[Pod]:
        """Create Deployment with specified replicas."""
        pods = []
        for i in range(replicas):
            pod = Pod(
                name=f"{name}-{uuid.uuid4().hex[:8]}",
                node="",
                containers=[image],
                cpu_request=cpu_request,
                memory_request=memory_request
            )
            if self.schedule_pod(pod):
                pods.append(pod)
        
        print(f"\n📦 Deployment '{name}' created: {len(pods)}/{replicas} pods running")
        return pods
    
    def get_cluster_status(self) -> Dict:
        """Get cluster resource utilization."""
        total_cpu = sum(n.cpu_capacity for n in self.nodes)
        used_cpu = sum(sum(p.cpu_request for p in n.pods if p.status == "Running") 
                       for n in self.nodes)
        total_memory = sum(n.memory_capacity for n in self.nodes)
        used_memory = sum(sum(p.memory_request for p in n.pods if p.status == "Running") 
                         for n in self.nodes)
        total_pods = sum(len(n.pods) for n in self.nodes)
        
        return {
            "total_nodes": len(self.nodes),
            "total_pods": total_pods,
            "cpu_utilization": (used_cpu / total_cpu) * 100 if total_cpu > 0 else 0,
            "memory_utilization": (used_memory / total_memory) * 100 if total_memory > 0 else 0,
            "total_cpu": total_cpu,
            "used_cpu": used_cpu,
            "total_memory": total_memory,
            "used_memory": used_memory
        }


@dataclass
class Service:
    """Represents a Kubernetes Service (stable endpoint for pods)."""
    name: str
    service_type: str = "ClusterIP"  # ClusterIP, NodePort, LoadBalancer
    cluster_ip: str = ""
    port: int = 8080
    target_pods: List[Pod] = field(default_factory=list)
    
    def __post_init__(self):
        if not self.cluster_ip:
            # Simulate cluster IP (10.96.x.y format)
            self.cluster_ip = f"10.96.{np.random.randint(0, 255)}.{np.random.randint(1, 255)}"
    
    def add_pod(self, pod: Pod):
        """Add pod to service backend."""
        self.target_pods.append(pod)
    
    def get_endpoint(self) -> Optional[Pod]:
        """Load balance request to healthy pod (round-robin simulation)."""
        healthy_pods = [p for p in self.target_pods if p.status == "Running"]
        if not healthy_pods:
            return None
        return np.random.choice(healthy_pods)
    
    def handle_request(self) -> Dict:
        """Simulate service request (load balance to pod)."""
        pod = self.get_endpoint()
        if not pod:
            return {"status": "error", "message": "No healthy pods available"}
        
        return {
            "status": "success",
            "service": self.name,
            "cluster_ip": self.cluster_ip,
            "pod_name": pod.name,
            "pod_ip": pod.ip,
            "node": pod.node
        }


# Example 1: Create Kubernetes cluster with 3 worker nodes
print("=" * 70)
print("EXAMPLE 1: Kubernetes Cluster Creation")
print("=" * 70)

cluster = KubernetesCluster(name="prod-ml-cluster")

# Add 3 worker nodes
cluster.add_node(Node(name="node-1", cpu_capacity=8.0, memory_capacity=16384))
cluster.add_node(Node(name="node-2", cpu_capacity=8.0, memory_capacity=16384))
cluster.add_node(Node(name="node-3", cpu_capacity=8.0, memory_capacity=16384))

print("\n" + "=" * 70)
print("EXAMPLE 2: Deploy ML Model Service (5 replicas)")
print("=" * 70)

# Create deployment: wafer yield predictor (5 replicas)
pods = cluster.create_deployment(
    name="wafer-yield-predictor",
    image="registry.io/wafer-model:v2.3",
    replicas=5,
    cpu_request=1.0,
    memory_request=1024
)

# Show cluster status
status = cluster.get_cluster_status()
print(f"\n📊 Cluster Status:")
print(f"   Nodes: {status['total_nodes']}")
print(f"   Pods: {status['total_pods']}")
print(f"   CPU Utilization: {status['cpu_utilization']:.1f}% ({status['used_cpu']:.1f}/{status['total_cpu']:.1f} cores)")
print(f"   Memory Utilization: {status['memory_utilization']:.1f}% ({status['used_memory']}/{status['total_memory']} MB)")

print("\n" + "=" * 70)
print("EXAMPLE 3: Create Service for Load Balancing")
print("=" * 70)

# Create Service (stable endpoint for pods)
service = Service(name="wafer-predictor-svc", port=8080)
for pod in pods:
    service.add_pod(pod)

print(f"✅ Service created: {service.name}")
print(f"   Type: {service.service_type}")
print(f"   Cluster IP: {service.cluster_ip}:{service.port}")
print(f"   Backend Pods: {len(service.target_pods)}")

# Simulate 10 requests to service (load balancing)
print(f"\n🔄 Simulating 10 requests to service {service.cluster_ip}:8080...")
pod_request_counts = {}

for i in range(10):
    response = service.handle_request()
    if response["status"] == "success":
        pod_name = response["pod_name"]
        pod_request_counts[pod_name] = pod_request_counts.get(pod_name, 0) + 1

print(f"\n📊 Request Distribution Across Pods:")
for pod_name, count in sorted(pod_request_counts.items()):
    print(f"   {pod_name}: {count} requests")

print("\n💡 Key Insight: Service distributes traffic across all healthy pods (load balancing)")

print("\n" + "=" * 70)
print("EXAMPLE 4: Self-Healing - Pod Restart")
print("=" * 70)

# Simulate pod failure and restart
failed_pod = pods[0]
print(f"❌ Pod {failed_pod.name} crashed (IP: {failed_pod.ip})")
print(f"   Kubernetes detects failure and restarts pod...")

old_ip = failed_pod.ip
failed_pod.restart()

print(f"✅ Pod restarted successfully")
print(f"   Old IP: {old_ip}")
print(f"   New IP: {failed_pod.ip}")
print(f"   Service automatically routes to new IP (no manual intervention)")

print("\n💡 Key Insight: Pods are ephemeral (IP changes), Service provides stable endpoint")

## 3. 📈 Horizontal Pod Autoscaling (HPA) for ML Workloads

**Purpose:** Automatically scale ML model replicas based on CPU, memory, or custom metrics (requests per second, queue depth).

**Key Points:**
- **HPA (Horizontal Pod Autoscaler):** Adjusts replica count based on metrics (5 → 50 pods during traffic spike)
- **Metrics:** CPU utilization (target: 70%), memory, custom metrics (RPS, latency, queue length)
- **Scale-up:** When CPU >70%, add pods (max: 50 replicas)
- **Scale-down:** When CPU <50%, remove pods (min: 2 replicas, gradual cooldown)
- **Target tracking:** Maintains target metric (e.g., 70% CPU across all pods)

**Why It Matters:**
- **Cost efficiency:** Scale down during low traffic (50 pods → 5 pods saves 90% infrastructure cost)
- **Performance:** Scale up during peaks (handle 10,000 RPS without degradation)
- **Automatic:** No manual intervention (responds to traffic in 30 seconds)
- **Business continuity:** Prevent overload (traffic spike doesn't crash service)

**Post-Silicon Application:**
STDF parser HPA: Normal load 100 files/hour (5 pods sufficient), wafer test spike 1000 files/hour (HPA scales to 50 pods in 2 minutes), after spike HPA gradually scales down to 5 pods (15-minute cooldown prevents flapping).

In [ ]:
# Horizontal Pod Autoscaler (HPA) Simulation

class HorizontalPodAutoscaler:
    """Simulates Kubernetes HPA for automatic scaling."""
    
    def __init__(self, deployment_name: str, min_replicas: int = 2, 
                 max_replicas: int = 50, target_cpu_percent: float = 70.0):
        self.deployment_name = deployment_name
        self.min_replicas = min_replicas
        self.max_replicas = max_replicas
        self.target_cpu_percent = target_cpu_percent
        self.current_replicas = min_replicas
        self.scale_history = []
        
    def calculate_desired_replicas(self, current_cpu_percent: float) -> int:
        """Calculate desired replica count based on current CPU utilization."""
        if current_cpu_percent == 0:
            return self.current_replicas
        
        # HPA formula: desired = ceil(current_replicas * (current_metric / target_metric))
        desired = int(np.ceil(self.current_replicas * (current_cpu_percent / self.target_cpu_percent)))
        
        # Apply min/max constraints
        desired = max(self.min_replicas, min(desired, self.max_replicas))
        
        return desired
    
    def scale(self, current_cpu_percent: float, timestamp: str) -> Dict:
        """Execute scaling decision based on current CPU."""
        desired = self.calculate_desired_replicas(current_cpu_percent)
        
        action = "no_change"
        if desired > self.current_replicas:
            action = "scale_up"
        elif desired < self.current_replicas:
            action = "scale_down"
        
        old_replicas = self.current_replicas
        self.current_replicas = desired
        
        scale_event = {
            "timestamp": timestamp,
            "current_cpu": current_cpu_percent,
            "target_cpu": self.target_cpu_percent,
            "old_replicas": old_replicas,
            "new_replicas": desired,
            "action": action
        }
        self.scale_history.append(scale_event)
        
        if action != "no_change":
            symbol = "📈" if action == "scale_up" else "📉"
            print(f"{symbol} {timestamp}: CPU {current_cpu_percent:.1f}% → "
                  f"Scale {old_replicas} → {desired} replicas ({action})")
        
        return scale_event
    
    def get_scaling_summary(self) -> Dict:
        """Summarize scaling activity."""
        scale_ups = sum(1 for e in self.scale_history if e["action"] == "scale_up")
        scale_downs = sum(1 for e in self.scale_history if e["action"] == "scale_down")
        max_replicas_used = max(e["new_replicas"] for e in self.scale_history) if self.scale_history else 0
        
        return {
            "total_events": len(self.scale_history),
            "scale_ups": scale_ups,
            "scale_downs": scale_downs,
            "max_replicas": max_replicas_used,
            "final_replicas": self.current_replicas
        }


def simulate_traffic_pattern(hours: int = 24) -> pd.DataFrame:
    """Simulate realistic traffic pattern (daily cycle with spikes)."""
    timestamps = pd.date_range(start="2024-01-01 00:00", periods=hours, freq="h")
    
    # Base traffic pattern (sinusoidal daily cycle)
    base_load = 50 + 30 * np.sin(np.linspace(0, 2*np.pi, hours))
    
    # Add random spikes
    spikes = np.zeros(hours)
    spike_hours = np.random.choice(hours, size=3, replace=False)
    for hour in spike_hours:
        spikes[hour] = np.random.uniform(40, 60)
    
    # Combine base + spikes + noise
    cpu_utilization = base_load + spikes + np.random.normal(0, 5, hours)
    cpu_utilization = np.clip(cpu_utilization, 10, 95)  # Keep within 10-95%
    
    return pd.DataFrame({
        "timestamp": timestamps,
        "cpu_percent": cpu_utilization
    })


# Example 1: Simulate 24-hour autoscaling with traffic pattern
print("=" * 70)
print("EXAMPLE 1: HPA in Action - 24 Hour Traffic Pattern")
print("=" * 70)

# Create HPA
hpa = HorizontalPodAutoscaler(
    deployment_name="wafer-yield-predictor",
    min_replicas=5,
    max_replicas=50,
    target_cpu_percent=70.0
)

print(f"📊 HPA Configuration:")
print(f"   Deployment: {hpa.deployment_name}")
print(f"   Min Replicas: {hpa.min_replicas}")
print(f"   Max Replicas: {hpa.max_replicas}")
print(f"   Target CPU: {hpa.target_cpu_percent}%")

# Generate traffic pattern
traffic_df = simulate_traffic_pattern(hours=24)

print(f"\n🔄 Starting autoscaling simulation (24 hours)...\n")

# Simulate HPA decisions every hour
for _, row in traffic_df.iterrows():
    hpa.scale(
        current_cpu_percent=row["cpu_percent"],
        timestamp=row["timestamp"].strftime("%Y-%m-%d %H:%M")
    )

# Get summary
summary = hpa.get_scaling_summary()
print(f"\n📊 Scaling Summary (24 hours):")
print(f"   Total Scaling Events: {summary['total_events']}")
print(f"   Scale-Ups: {summary['scale_ups']}")
print(f"   Scale-Downs: {summary['scale_downs']}")
print(f"   Max Replicas Used: {summary['max_replicas']}")
print(f"   Final Replicas: {summary['final_replicas']}")

# Visualize autoscaling behavior
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot 1: CPU utilization over time
ax1.plot(traffic_df["timestamp"], traffic_df["cpu_percent"], 
         linewidth=2, label="CPU Utilization", color='#FF6B6B')
ax1.axhline(y=hpa.target_cpu_percent, color='green', linestyle='--', 
            linewidth=2, label=f'Target ({hpa.target_cpu_percent}%)')
ax1.fill_between(traffic_df["timestamp"], 0, traffic_df["cpu_percent"], 
                  alpha=0.3, color='#FF6B6B')
ax1.set_ylabel("CPU Utilization (%)", fontsize=12, fontweight='bold')
ax1.set_title("Horizontal Pod Autoscaling - CPU and Replica Count", 
              fontsize=14, fontweight='bold', pad=20)
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 100)

# Plot 2: Replica count over time
replica_counts = [e["new_replicas"] for e in hpa.scale_history]
timestamps = [pd.to_datetime(e["timestamp"]) for e in hpa.scale_history]

ax2.plot(timestamps, replica_counts, linewidth=2.5, 
         marker='o', markersize=6, label="Replica Count", color='#4ECDC4')
ax2.fill_between(timestamps, 0, replica_counts, alpha=0.3, color='#4ECDC4')
ax2.axhline(y=hpa.min_replicas, color='orange', linestyle=':', 
            linewidth=2, label=f'Min ({hpa.min_replicas})')
ax2.axhline(y=hpa.max_replicas, color='red', linestyle=':', 
            linewidth=2, label=f'Max ({hpa.max_replicas})')
ax2.set_xlabel("Time", fontsize=12, fontweight='bold')
ax2.set_ylabel("Pod Replicas", fontsize=12, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, hpa.max_replicas + 5)

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("   • HPA scales UP when CPU > 70% (handle increased load)")
print("   • HPA scales DOWN when CPU < 70% (save costs)")
print("   • Scaling is gradual (prevents flapping)")
print("   • Min/max constraints prevent extreme scaling")

print("\n" + "=" * 70)
print("EXAMPLE 2: Cost Savings from Autoscaling")
print("=" * 70)

# Calculate cost savings
avg_replicas_static = 50  # Static deployment (always 50 pods for peak)
avg_replicas_hpa = np.mean(replica_counts)
cost_per_pod_hour = 0.05  # $0.05/pod/hour

static_cost_24h = avg_replicas_static * 24 * cost_per_pod_hour
hpa_cost_24h = avg_replicas_hpa * 24 * cost_per_pod_hour
savings_24h = static_cost_24h - hpa_cost_24h
savings_percent = (savings_24h / static_cost_24h) * 100

print(f"💰 Cost Analysis (24 hours):")
print(f"   Static Deployment (50 pods always):")
print(f"      Cost: ${static_cost_24h:.2f}")
print(f"   ")
print(f"   HPA Deployment (avg {avg_replicas_hpa:.1f} pods):")
print(f"      Cost: ${hpa_cost_24h:.2f}")
print(f"   ")
print(f"   Savings: ${savings_24h:.2f} ({savings_percent:.1f}%)")
print(f"   Monthly Savings: ${savings_24h * 30:.2f}")
print(f"   Annual Savings: ${savings_24h * 365:.2f}")

print("\n💡 Business Impact:")
print(f"   • Same performance during peaks (50 pods available)")
print(f"   • {savings_percent:.1f}% cost reduction (scale down during low traffic)")
print(f"   • Fully automated (no manual scaling required)")

## 4. 🔄 Rolling Updates and Rollbacks - Zero-Downtime Deployments

**Purpose:** Update ML models to new versions without service interruption (v2.3 → v2.4 with zero dropped requests).

**Key Points:**
- **Rolling Update Strategy:** Update pods gradually (1 at a time, wait for readiness before next)
- **MaxUnavailable:** Maximum pods down during update (1 out of 10 = 90% capacity maintained)
- **MaxSurge:** Extra pods during update (surge=1 means 11 pods temporarily, smooth transition)
- **Readiness Probe:** Health check determines when new pod ready (wait 30s after startup)
- **Rollback:** Revert to previous version instantly (`kubectl rollout undo`)

**Why It Matters:**
- **Zero downtime:** Users don't notice deployment (service continuously available)
- **Gradual rollout:** Canary pattern (update 1 pod, monitor, then rest)
- **Safety:** Rollback in 30 seconds if new version has bugs
- **Version history:** Keep last 10 revisions (audit trail, easy rollback)

**Post-Silicon Application:**
Wafer yield predictor v2.3 → v2.4 deployment: 10 pods total, rolling update with maxUnavailable=1, update 1 pod every 2 minutes (monitor error rate), total deployment time 20 minutes, if v2.4 error rate >1%, automatic rollback to v2.3 in 30 seconds.

In [ ]:
# Rolling Update Simulation

class RollingUpdateDeployment:
    """Simulates Kubernetes rolling update strategy."""
    
    def __init__(self, name: str, replicas: int, image: str, 
                 max_unavailable: int = 1, max_surge: int = 1):
        self.name = name
        self.desired_replicas = replicas
        self.image = image
        self.max_unavailable = max_unavailable
        self.max_surge = max_surge
        self.pods: List[Pod] = []
        self.revision_history = []
        
        # Create initial pods
        for i in range(replicas):
            self.pods.append(Pod(
                name=f"{name}-{uuid.uuid4().hex[:8]}",
                node=f"node-{(i % 3) + 1}",
                containers=[image]
            ))
        
        self.revision_history.append({
            "revision": 1,
            "image": image,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
    
    def update_to_version(self, new_image: str, simulate_delay: bool = False) -> Dict:
        """Perform rolling update to new image version."""
        print(f"\n🔄 Starting rolling update: {self.image} → {new_image}")
        print(f"   Strategy: maxUnavailable={self.max_unavailable}, maxSurge={self.max_surge}")
        
        update_log = []
        old_pods = self.pods.copy()
        new_pods = []
        
        # Create surge pods first (extra capacity during update)
        surge_count = min(self.max_surge, self.desired_replicas)
        for i in range(surge_count):
            new_pod = Pod(
                name=f"{self.name}-{uuid.uuid4().hex[:8]}",
                node=f"node-{np.random.randint(1, 4)}",
                containers=[new_image]
            )
            new_pods.append(new_pod)
            self.pods.append(new_pod)
            update_log.append(f"✅ Created new pod {new_pod.name} ({new_image})")
            if simulate_delay:
                time.sleep(0.1)
        
        # Rolling update: replace old pods one by one
        for i, old_pod in enumerate(old_pods):
            # Terminate old pod
            old_pod.status = "Terminating"
            update_log.append(f"🔻 Terminating old pod {old_pod.name} ({self.image})")
            
            # If we haven't created enough surge pods, create new pod now
            if i >= surge_count:
                new_pod = Pod(
                    name=f"{self.name}-{uuid.uuid4().hex[:8]}",
                    node=old_pod.node,  # Schedule on same node
                    containers=[new_image]
                )
                new_pods.append(new_pod)
                update_log.append(f"✅ Created new pod {new_pod.name} ({new_image})")
                if simulate_delay:
                    time.sleep(0.1)
            
            # Wait for readiness probe (simulated)
            update_log.append(f"⏳ Waiting for readiness probe...")
            if simulate_delay:
                time.sleep(0.05)
            
            # Remove old pod
            self.pods.remove(old_pod)
            
            # Print progress
            progress = ((i + 1) / len(old_pods)) * 100
            print(f"   Progress: {progress:.0f}% ({i+1}/{len(old_pods)} pods updated)")
        
        # Remove extra surge pods
        while len(self.pods) > self.desired_replicas:
            surplus_pod = self.pods.pop()
            update_log.append(f"🔻 Removing surge pod {surplus_pod.name}")
        
        # Update image and revision history
        old_image = self.image
        self.image = new_image
        self.revision_history.append({
            "revision": len(self.revision_history) + 1,
            "image": new_image,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
        
        print(f"\n✅ Rolling update completed successfully")
        print(f"   Old version: {old_image}")
        print(f"   New version: {new_image}")
        print(f"   Pods updated: {len(old_pods)}")
        print(f"   Final pod count: {len(self.pods)}")
        
        return {
            "status": "success",
            "old_image": old_image,
            "new_image": new_image,
            "pods_updated": len(old_pods),
            "update_log": update_log
        }
    
    def rollback(self) -> Dict:
        """Rollback to previous revision."""
        if len(self.revision_history) < 2:
            return {"status": "error", "message": "No previous revision to rollback to"}
        
        current_revision = self.revision_history[-1]
        previous_revision = self.revision_history[-2]
        
        print(f"\n⏪ Rolling back deployment...")
        print(f"   Current: {current_revision['image']} (revision {current_revision['revision']})")
        print(f"   Target: {previous_revision['image']} (revision {previous_revision['revision']})")
        
        # Perform rollback (update to previous image)
        result = self.update_to_version(previous_revision['image'])
        
        print(f"\n✅ Rollback completed in 30 seconds")
        return {
            "status": "success",
            "reverted_from": current_revision['image'],
            "reverted_to": previous_revision['image']
        }
    
    def get_revision_history(self) -> List[Dict]:
        """Get deployment revision history."""
        return self.revision_history


# Example 1: Rolling update simulation
print("=" * 70)
print("EXAMPLE 1: Rolling Update - v2.3 → v2.4")
print("=" * 70)

# Create deployment with v2.3
deployment = RollingUpdateDeployment(
    name="wafer-yield-predictor",
    replicas=10,
    image="registry.io/wafer-model:v2.3",
    max_unavailable=1,
    max_surge=1
)

print(f"📦 Initial Deployment:")
print(f"   Name: {deployment.name}")
print(f"   Replicas: {len(deployment.pods)}")
print(f"   Image: {deployment.image}")

# Perform rolling update to v2.4
result = deployment.update_to_version("registry.io/wafer-model:v2.4", simulate_delay=True)

print("\n" + "=" * 70)
print("EXAMPLE 2: Revision History")
print("=" * 70)

history = deployment.get_revision_history()
print(f"📜 Deployment Revision History:")
for rev in history:
    print(f"   Revision {rev['revision']}: {rev['image']} (deployed at {rev['timestamp']})")

print("\n" + "=" * 70)
print("EXAMPLE 3: Rollback to Previous Version")
print("=" * 70)

# Simulate bug in v2.4 - rollback to v2.3
print("❌ Bug detected in v2.4 (error rate increased to 5%)")
print("🚨 Initiating automatic rollback...")

rollback_result = deployment.rollback()

print(f"\n📊 Post-Rollback Status:")
print(f"   Current Image: {deployment.image}")
print(f"   Active Pods: {len(deployment.pods)}")
print(f"   All pods healthy: ✅")

print("\n💡 Key Insights:")
print("   • Rolling update: Zero downtime (always ≥9 pods running)")
print("   • MaxUnavailable=1: 90% capacity during update (9/10 pods)")
print("   • MaxSurge=1: Temporarily 11 pods (smooth transition)")
print("   • Rollback: 30 seconds to revert (vs hours of manual rollback)")
print("   • Version history: Audit trail for compliance")

# Visualize rolling update timeline
print("\n" + "=" * 70)
print("EXAMPLE 4: Rolling Update Timeline Visualization")
print("=" * 70)

# Simulate detailed timeline
timeline_data = {
    "time_seconds": [0, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300],
    "v2.3_pods": [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0],
    "v2.4_pods": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "total_pods": [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
}

timeline_df = pd.DataFrame(timeline_data)

fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(timeline_df["time_seconds"], timeline_df["v2.3_pods"], 
        marker='o', linewidth=3, markersize=8, label="v2.3 Pods (Old)", color='#FF6B6B')
ax.plot(timeline_df["time_seconds"], timeline_df["v2.4_pods"], 
        marker='s', linewidth=3, markersize=8, label="v2.4 Pods (New)", color='#4ECDC4')
ax.plot(timeline_df["time_seconds"], timeline_df["total_pods"], 
        linestyle='--', linewidth=2.5, label="Total Pods", color='#95E1D3')

ax.fill_between(timeline_df["time_seconds"], 0, timeline_df["v2.3_pods"], 
                 alpha=0.3, color='#FF6B6B')
ax.fill_between(timeline_df["time_seconds"], 0, timeline_df["v2.4_pods"], 
                 alpha=0.3, color='#4ECDC4')

ax.set_xlabel("Time (seconds)", fontsize=12, fontweight='bold')
ax.set_ylabel("Number of Pods", fontsize=12, fontweight='bold')
ax.set_title("Rolling Update Timeline - Zero Downtime Deployment", 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='center right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 12)

# Add annotations
ax.annotate('Update Start', xy=(0, 10), xytext=(20, 11),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=10, fontweight='bold')
ax.annotate('50% Complete', xy=(150, 5), xytext=(100, 8),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=10, fontweight='bold')
ax.annotate('Update Complete', xy=(300, 10), xytext=(240, 11),
            arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
            fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("💡 Timeline Analysis:")
print("   • Duration: 5 minutes (300 seconds)")
print("   • Pod replacement rate: 1 pod every 30 seconds")
print("   • Service availability: 100% (always 10 pods running)")
print("   • Traffic distribution: Gradual shift from v2.3 → v2.4")

## 5. 🚀 Real-World Project Templates

---

### Project 1: Auto-Scaling STDF Parser Service on Kubernetes

**Objective:** Deploy STDF binary file parser on Kubernetes with HPA for auto-scaling (5-50 pods based on queue depth).

**Business Value:**
- **Cost efficiency:** Scale down to 5 pods during off-hours (save 90% infrastructure cost)
- **Performance:** Handle wafer test spikes (1000 files/hour) without latency degradation
- **Reliability:** 3 availability zones, survive datacenter failures

**Features to Implement:**
- **Deployment:** 5 replicas initially, HPA target: 80% CPU or 100 messages in SQS queue
- **Service:** LoadBalancer type (external IP for test equipment)
- **ConfigMap:** Parser configuration (chunk_size, output_format, compression)
- **Secret:** Database credentials (PostgreSQL connection string)
- **PersistentVolume:** Store parsed data temporarily (1TB SSD)
- **Resource limits:** CPU: 1 core, Memory: 2GB per pod

**Success Criteria:**
- ✅ Auto-scale from 5 → 50 pods in 3 minutes during spike
- ✅ Parse latency <2 seconds per STDF file (10K devices)
- ✅ 99.9% uptime (8.76 hours downtime/year max)
- ✅ Zero data loss (persistent queue, retries)
- ✅ Cost <$500/month (vs $5000/month static 50 pods)

**Kubernetes YAML Skeleton:**
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: stdf-parser
spec:
  replicas: 5
  template:
    spec:
      containers:
      - name: parser
        image: registry.io/stdf-parser:v1.2
        resources:
          requests:
            cpu: "1"
            memory: "2Gi"
        env:
        - name: DB_CONNECTION
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: connection-string
---
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: stdf-parser-hpa
spec:
  scaleTargetRef:
    kind: Deployment
    name: stdf-parser
  minReplicas: 5
  maxReplicas: 50
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 80
```

---

### Project 2: High-Availability ML Model API with Multi-Region Deployment

**Objective:** Deploy wafer yield prediction API across 3 AWS regions (us-west-2, us-east-1, eu-west-1) with global load balancing.

**Business Value:**
- **Latency:** <50ms response time worldwide (serve from nearest region)
- **Availability:** 99.99% uptime (4 regions, survive region failures)
- **Compliance:** EU data stays in EU (GDPR compliance)

**Features to Implement:**
- **Multi-cluster:** 3 Kubernetes clusters (EKS in us-west-2, us-east-1, eu-west-1)
- **Global load balancer:** AWS Route53 geo-routing (direct traffic to nearest region)
- **StatefulSet:** Model pods with persistent model storage (EBS volumes)
- **Cross-region replication:** Model updates replicate to all regions (S3 sync)
- **Health checks:** Readiness/liveness probes, Route53 health checks
- **Monitoring:** Prometheus federated across regions, Grafana global dashboard

**Success Criteria:**
- ✅ Latency: <50ms p99 from any location
- ✅ Failover: <10 seconds when region fails
- ✅ Model sync: New version deployed to all regions within 15 minutes
- ✅ Cost: <$2000/month (3 regions, auto-scaling)

**Architecture:**
```
               ┌─────────────────────┐
               │ Route53 Global LB   │
               └──────────┬──────────┘
                          │
        ┌─────────────────┼─────────────────┐
        │                 │                 │
   ┌────▼────┐       ┌────▼────┐       ┌────▼────┐
   │ us-west-2│       │us-east-1│       │ eu-west-1│
   │   EKS   │       │   EKS   │       │   EKS   │
   │ 10 pods │       │ 10 pods │       │ 10 pods │
   └─────────┘       └─────────┘       └─────────┘
```

---

### Project 3: GPU-Accelerated Deep Learning Training on Kubernetes

**Objective:** Run distributed PyTorch training on Kubernetes with 4 NVIDIA V100 GPUs (wafer defect detection model).

**Business Value:**
- **Training speed:** 10 hours → 2.5 hours (4 GPUs, distributed training)
- **Cost efficiency:** Spot instances for 70% discount (save $5000/month)
- **Scalability:** Auto-provision GPUs, terminate when training complete

**Features to Implement:**
- **Job:** Kubernetes Job for training (not Deployment, terminates when complete)
- **GPU scheduling:** Request 4 GPUs (nvidia.com/gpu: 4)
- **Distributed training:** PyTorch DistributedDataParallel (DDP)
- **Node affinity:** Schedule on p3.8xlarge instances (4 V100 GPUs)
- **Volume:** Mount training data from EFS (100GB SEM images)
- **Checkpointing:** Save checkpoints to S3 every epoch (resume on failure)
- **TensorBoard:** Service for monitoring training (port-forward for access)

**Success Criteria:**
- ✅ Training time: <3 hours (vs 10 hours single GPU)
- ✅ GPU utilization: >90% (efficient data pipeline)
- ✅ Cost: <$50 per training run (spot instances)
- ✅ Automatic retry: Resume from checkpoint on preemption
- ✅ Model accuracy: >98% (detect scratches, particles, voids)

**Kubernetes Job YAML:**
```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: wafer-defect-training
spec:
  parallelism: 1  # Single job, 4 GPUs
  template:
    spec:
      restartPolicy: OnFailure
      containers:
      - name: pytorch-training
        image: pytorch/pytorch:2.0.0-cuda11.8-cudnn8
        command: ["python", "train.py"]
        resources:
          limits:
            nvidia.com/gpu: 4
        volumeMounts:
        - name: training-data
          mountPath: /data
        - name: checkpoints
          mountPath: /checkpoints
      volumes:
      - name: training-data
        persistentVolumeClaim:
          claimName: efs-training-data
      - name: checkpoints
        emptyDir: {}
      nodeSelector:
        node.kubernetes.io/instance-type: p3.8xlarge
```

---

### Project 4: Batch Processing Pipeline with Kubernetes CronJobs

**Objective:** Nightly wafer map analysis (process 1000 wafers, generate spatial correlation reports, email stakeholders).

**Business Value:**
- **Automation:** Zero manual effort (runs every night at 2 AM)
- **Cost:** Only pay for compute during job execution (terminate after completion)
- **Scalability:** Process 1000 wafers in parallel (100 concurrent pods)

**Features to Implement:**
- **CronJob:** Schedule daily at 2 AM UTC (`0 2 * * *`)
- **Parallel Jobs:** 100 parallel pods (each processes 10 wafers)
- **Job completion:** Terminate pods when all wafers processed
- **Output:** Write reports to S3, send email via SES
- **Failure handling:** Retry failed wafers up to 3 times
- **Resource limits:** CPU: 2 cores, Memory: 4GB per pod
- **Spot instances:** Use spot nodes for 70% cost savings

**Success Criteria:**
- ✅ Processing time: <2 hours for 1000 wafers (vs 20 hours sequential)
- ✅ Parallelism: 100 concurrent pods (elastic scaling)
- ✅ Cost: <$20 per run (spot instances, terminate after completion)
- ✅ Reliability: 99.9% success rate (automatic retries)
- ✅ Alerting: Email notification on completion or failure

**CronJob YAML:**
```yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: nightly-wafer-analysis
spec:
  schedule: "0 2 * * *"  # 2 AM UTC daily
  jobTemplate:
    spec:
      parallelism: 100
      completions: 100
      template:
        spec:
          restartPolicy: OnFailure
          containers:
          - name: wafer-analyzer
            image: registry.io/wafer-analyzer:v1.0
            args: ["--batch-size", "10"]
            resources:
              requests:
                cpu: "2"
                memory: "4Gi"
```

---

### Project 5: Blue-Green Deployment for Risk-Free Model Updates

**Objective:** Deploy model updates with instant rollback capability (blue environment v2.3, green environment v2.4, switch traffic in 10 seconds).

**Business Value:**
- **Risk mitigation:** Test v2.4 in production without affecting users (blue still serves traffic)
- **Instant rollback:** Revert in 10 seconds if v2.4 has issues (vs 30 minutes rolling update rollback)
- **A/B testing:** Send 10% traffic to green, compare metrics, then switch 100%

**Features to Implement:**
- **Two Deployments:** blue (v2.3) and green (v2.4) running simultaneously
- **Service:** Switch traffic via selector label (version: blue → version: green)
- **Resource duplication:** 2x infrastructure temporarily (10 blue pods + 10 green pods)
- **Smoke tests:** Automated tests on green before switching traffic
- **Metrics comparison:** Compare error rate, latency between blue and green
- **Quick switch:** Update Service selector label (`kubectl patch`)

**Success Criteria:**
- ✅ Deployment time: 15 minutes (deploy green, test, switch)
- ✅ Rollback time: 10 seconds (change Service selector)
- ✅ Zero downtime: 100% (always 10 pods serving traffic)
- ✅ Cost: 2x infrastructure for 15 minutes only
- ✅ Confidence: Full production testing before switch

**Blue-Green Switch Process:**
```bash
# Step 1: Deploy green (v2.4) alongside blue (v2.3)
kubectl apply -f deployment-green.yaml

# Step 2: Run smoke tests on green
curl http://green-service:8080/health
curl http://green-service:8080/predict -d '{"features": [...]}'

# Step 3: Compare metrics (Prometheus)
# Blue: Error rate 0.1%, Latency 45ms p99
# Green: Error rate 0.1%, Latency 42ms p99 ✅

# Step 4: Switch traffic (update Service selector)
kubectl patch service wafer-predictor -p '{"spec":{"selector":{"version":"green"}}}'

# Traffic now flows to green (v2.4)
# Blue (v2.3) still running, instant rollback if needed

# Step 5: If green is stable after 1 hour, delete blue
kubectl delete deployment wafer-predictor-blue
```

---

### Project 6: StatefulSet for Model Caching Layer with Redis

**Objective:** Deploy Redis cluster on Kubernetes for caching model predictions (reduce redundant inference by 80%).

**Business Value:**
- **Latency:** 2ms cache hit vs 50ms model inference (25x speedup)
- **Cost:** Reduce inference load (80% cache hit rate, 5x fewer model pods needed)
- **Scalability:** Handle 50,000 RPS (cache absorbs load)

**Features to Implement:**
- **StatefulSet:** 3 Redis pods with stable network identities (redis-0, redis-1, redis-2)
- **PersistentVolumes:** Each Redis pod gets 50GB SSD (data survives pod restart)
- **Headless Service:** DNS entries for each pod (redis-0.redis.svc.cluster.local)
- **Redis Cluster:** Sharded data across 3 pods (automatic failover)
- **TTL:** Cache entries expire after 1 hour (fresh predictions)
- **Monitoring:** Redis Exporter for Prometheus (hit rate, memory usage)

**Success Criteria:**
- ✅ Cache hit rate: >80% (most predictions cached)
- ✅ Latency: <5ms p99 for cache hits
- ✅ Availability: 99.99% (Redis cluster, automatic failover)
- ✅ Cost savings: 80% reduction in model inference (5 model pods vs 25 pods)

**StatefulSet YAML:**
```yaml
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: redis
spec:
  serviceName: redis
  replicas: 3
  selector:
    matchLabels:
      app: redis
  template:
    metadata:
      labels:
        app: redis
    spec:
      containers:
      - name: redis
        image: redis:7-alpine
        ports:
        - containerPort: 6379
        volumeMounts:
        - name: redis-data
          mountPath: /data
  volumeClaimTemplates:
  - metadata:
      name: redis-data
    spec:
      accessModes: ["ReadWriteOnce"]
      resources:
        requests:
          storage: 50Gi
```

---

### Project 7: Kubernetes Ingress for External Access and TLS Termination

**Objective:** Expose ML API to internet with HTTPS, domain name (api.wafer-validation.com), rate limiting, and authentication.

**Business Value:**
- **Security:** TLS encryption (HTTPS), prevent eavesdropping
- **Usability:** Custom domain (vs random IP address)
- **Protection:** Rate limiting (1000 req/min per IP), prevent abuse
- **Cost:** Single load balancer for all services (vs separate LB per service)

**Features to Implement:**
- **Ingress:** NGINX Ingress Controller
- **TLS Certificate:** Let's Encrypt auto-renewal (free certificates)
- **Domain:** api.wafer-validation.com → Service
- **Rate limiting:** 1000 requests/minute per IP
- **Authentication:** API key validation (Ingress middleware)
- **Path routing:** /predict → model service, /parse → STDF parser service

**Success Criteria:**
- ✅ HTTPS: All traffic encrypted (A+ SSL Labs rating)
- ✅ Auto-renewal: Certificates renew before expiry
- ✅ Rate limiting: Block IPs exceeding 1000 req/min
- ✅ Cost: $20/month (single load balancer for all services)

**Ingress YAML:**
```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: ml-api-ingress
  annotations:
    cert-manager.io/cluster-issuer: letsencrypt-prod
    nginx.ingress.kubernetes.io/rate-limit: "1000"
spec:
  tls:
  - hosts:
    - api.wafer-validation.com
    secretName: api-tls-cert
  rules:
  - host: api.wafer-validation.com
    http:
      paths:
      - path: /predict
        pathType: Prefix
        backend:
          service:
            name: wafer-predictor
            port:
              number: 8080
      - path: /parse
        pathType: Prefix
        backend:
          service:
            name: stdf-parser
            port:
              number: 8080
```

---

### Project 8: Kubernetes Monitoring Stack with Prometheus and Grafana

**Objective:** Complete observability for ML workloads (metrics, logs, traces, dashboards, alerting).

**Business Value:**
- **Visibility:** Real-time metrics (request rate, latency, error rate, resource usage)
- **Alerting:** PagerDuty notification when error rate >1% or latency >100ms
- **Troubleshooting:** Correlate logs, metrics, traces (find root cause in 5 minutes vs 2 hours)
- **Capacity planning:** Historical trends (predict when to scale infrastructure)

**Features to Implement:**
- **Prometheus:** Scrape metrics from all pods (CPU, memory, custom metrics)
- **Grafana:** Dashboards for ML metrics (predictions/sec, accuracy, latency percentiles)
- **Alertmanager:** Route alerts to PagerDuty, Slack, email
- **Loki:** Log aggregation (ELK alternative, Grafana integration)
- **Jaeger:** Distributed tracing (track requests across microservices)
- **Dashboards:** Pre-built for Kubernetes, ML models, business metrics

**Success Criteria:**
- ✅ Metrics retention: 90 days (Prometheus + Thanos)
- ✅ Dashboard count: 10+ (cluster health, model performance, business KPIs)
- ✅ Alert latency: <1 minute (from incident to notification)
- ✅ Cost: <$100/month (storage, compute for monitoring stack)

**Monitoring Architecture:**
```
┌──────────────────────────────────────────────┐
│           Kubernetes Cluster                 │
│                                              │
│  ┌────────┐  ┌────────┐  ┌────────┐        │
│  │ Pod 1  │  │ Pod 2  │  │ Pod 3  │        │
│  │ /metrics│  │ /metrics│  │ /metrics│        │
│  └───┬────┘  └───┬────┘  └───┬────┘        │
│      │           │           │              │
│      └───────────┼───────────┘              │
│                  │                          │
│          ┌───────▼────────┐                 │
│          │  Prometheus    │                 │
│          │  (Scrape       │                 │
│          │   every 15s)   │                 │
│          └───────┬────────┘                 │
│                  │                          │
│          ┌───────▼────────┐                 │
│          │   Grafana      │                 │
│          │  (Dashboards)  │                 │
│          └───────┬────────┘                 │
│                  │                          │
│          ┌───────▼────────┐                 │
│          │ Alertmanager   │                 │
│          │ → PagerDuty    │                 │
│          └────────────────┘                 │
└──────────────────────────────────────────────┘
```

## 6. 📋 Comprehensive Takeaways - Kubernetes ML Mastery

---

### Section 1: Kubernetes Architecture Fundamentals

**Control Plane Components:**
- **API Server (kube-apiserver):** Entry point for all operations (kubectl → API server)
- **Scheduler (kube-scheduler):** Assigns pods to nodes (bin packing, resource constraints)
- **Controller Manager:** Maintains desired state (5 replicas specified → ensures 5 running)
- **etcd:** Distributed key-value store (cluster state, configuration)

**Worker Node Components:**
- **kubelet:** Agent on each node (starts/stops pods, reports status)
- **kube-proxy:** Network proxy (implements Services, load balancing)
- **Container Runtime:** Runs containers (Docker, containerd, CRI-O)

**Key Concepts:**
- **Pod:** Smallest deployable unit (1+ containers, shared network/storage, ephemeral)
- **Deployment:** Manages replicas (declarative, rolling updates, rollbacks)
- **Service:** Stable networking (ClusterIP, NodePort, LoadBalancer types)
- **Namespace:** Virtual cluster (dev, staging, production isolation)

**Why This Architecture:**
- **Declarative:** Specify desired state (5 replicas), Kubernetes maintains it
- **Self-healing:** Pod crashes → kubelet restarts, node fails → scheduler reschedules
- **Scalable:** Control plane handles 1000s of nodes, 100,000s of pods

---

### Section 2: Pod Lifecycle and Health Checks

**Pod Phases:**
1. **Pending:** Waiting to be scheduled (no node available yet)
2. **Running:** Pod scheduled, containers started
3. **Succeeded:** All containers exited successfully (Job completed)
4. **Failed:** At least one container exited with error
5. **Unknown:** Node communication lost (kubelet not responding)

**Probes (Health Checks):**
```yaml
livenessProbe:  # Is container running? (Restart if fails)
  httpGet:
    path: /health
    port: 8080
  initialDelaySeconds: 30
  periodSeconds: 10
  
readinessProbe:  # Is container ready for traffic? (Remove from Service if fails)
  httpGet:
    path: /ready
    port: 8080
  initialDelaySeconds: 10
  periodSeconds: 5
```

**Restart Policies:**
- **Always:** Default, always restart (for long-running services)
- **OnFailure:** Restart only if exits with error (for Jobs)
- **Never:** Never restart (for one-time tasks)

**Best Practices:**
- ✅ **Liveness probe:** Detect deadlocks (app running but unresponsive)
- ✅ **Readiness probe:** Prevent routing to pods during startup (wait for model loading)
- ✅ **Separate probes:** Different endpoints (liveness: /health, readiness: /ready)
- ✅ **Graceful shutdown:** Handle SIGTERM (finish in-flight requests before exit)

---

### Section 3: Services and Networking

**Service Types:**
```yaml
# ClusterIP (default): Internal only (pod-to-pod communication)
apiVersion: v1
kind: Service
metadata:
  name: model-internal
spec:
  type: ClusterIP
  selector:
    app: model
  ports:
  - port: 8080
    targetPort: 8080

# NodePort: External access via <NodeIP>:<NodePort>
apiVersion: v1
kind: Service
metadata:
  name: model-external
spec:
  type: NodePort
  selector:
    app: model
  ports:
  - port: 8080
    targetPort: 8080
    nodePort: 30080  # Accessible at any-node-ip:30080

# LoadBalancer: Cloud load balancer (AWS ELB, GCP LB)
apiVersion: v1
kind: Service
metadata:
  name: model-lb
spec:
  type: LoadBalancer
  selector:
    app: model
  ports:
  - port: 80
    targetPort: 8080
```

**Service Discovery:**
- **DNS:** Every Service gets DNS name (`model-svc.default.svc.cluster.local`)
- **Environment Variables:** Injected into pods (`MODEL_SVC_SERVICE_HOST=10.96.1.5`)

**Load Balancing:**
- **Round-robin:** Default (distribute evenly across pods)
- **Session affinity:** Sticky sessions (`sessionAffinity: ClientIP`)

**Key Insight:**
Pods have dynamic IPs (change on restart), Services provide stable endpoint. Always use Service DNS name, never hardcode Pod IP.

---

### Section 4: Deployments and ReplicaSets

**Deployment Structure:**
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: wafer-model
spec:
  replicas: 10
  selector:
    matchLabels:
      app: wafer-model
  template:  # Pod template
    metadata:
      labels:
        app: wafer-model
    spec:
      containers:
      - name: model
        image: registry.io/wafer-model:v2.3
        resources:
          requests:  # Guaranteed resources
            cpu: "1"
            memory: "2Gi"
          limits:  # Maximum allowed
            cpu: "2"
            memory: "4Gi"
```

**Rolling Update Strategy:**
```yaml
strategy:
  type: RollingUpdate
  rollingUpdate:
    maxUnavailable: 1  # Max pods down during update
    maxSurge: 1        # Extra pods during update
```

**Update Commands:**
```bash
# Update image
kubectl set image deployment/wafer-model model=registry.io/wafer-model:v2.4

# Check rollout status
kubectl rollout status deployment/wafer-model

# View rollout history
kubectl rollout history deployment/wafer-model

# Rollback to previous version
kubectl rollout undo deployment/wafer-model

# Rollback to specific revision
kubectl rollout undo deployment/wafer-model --to-revision=3

# Pause rollout (canary deployment)
kubectl rollout pause deployment/wafer-model

# Resume rollout
kubectl rollout resume deployment/wafer-model
```

**ReplicaSet:**
- Deployment creates ReplicaSet (Deployment → ReplicaSet → Pods)
- ReplicaSet ensures desired replica count (5 pods specified → maintains 5 running)
- Don't create ReplicaSets directly (use Deployments for version management)

---

### Section 5: Horizontal Pod Autoscaling (HPA)

**HPA Metrics:**
```yaml
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: wafer-model-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: wafer-model
  minReplicas: 5
  maxReplicas: 50
  metrics:
  # CPU-based scaling
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 70
  # Memory-based scaling
  - type: Resource
    resource:
      name: memory
      target:
        type: Utilization
        averageUtilization: 80
  # Custom metric (Prometheus)
  - type: Pods
    pods:
      metric:
        name: http_requests_per_second
      target:
        type: AverageValue
        averageValue: "1000"
```

**HPA Formula:**
```
desired_replicas = ceil(current_replicas * (current_metric / target_metric))

Example:
current_replicas = 10
current_cpu = 85%
target_cpu = 70%
desired_replicas = ceil(10 * (85 / 70)) = ceil(12.14) = 13
```

**Scale-Down Cooldown:**
- Default: 5 minutes (prevent flapping)
- Gradual scale-down (don't drop from 50 → 5 instantly)

**Best Practices:**
- ✅ **Set appropriate targets:** CPU 70-80% (not 90%, leaves no buffer)
- ✅ **Define min/max:** Prevent scaling to 0 or infinity
- ✅ **Use custom metrics:** Better than CPU (e.g., queue depth, RPS)
- ✅ **Monitor scaling events:** Alert on frequent scaling (indicates instability)

---

### Section 6: ConfigMaps and Secrets

**ConfigMaps (Non-Sensitive Configuration):**
```yaml
# Create ConfigMap
apiVersion: v1
kind: ConfigMap
metadata:
  name: model-config
data:
  model_path: "/models/yield_v2.3.pkl"
  batch_size: "32"
  confidence_threshold: "0.85"
  feature_names: |
    Vdd
    Idd
    frequency
    temperature
    power
    yield

# Use in Pod
spec:
  containers:
  - name: model
    env:
    - name: MODEL_PATH
      valueFrom:
        configMapKeyRef:
          name: model-config
          key: model_path
    # Or mount as volume
    volumeMounts:
    - name: config
      mountPath: /config
  volumes:
  - name: config
    configMap:
      name: model-config
```

**Secrets (Sensitive Data):**
```yaml
# Create Secret
apiVersion: v1
kind: Secret
metadata:
  name: db-credentials
type: Opaque
data:
  username: YWRtaW4=  # Base64: admin
  password: cGFzc3dvcmQxMjM=  # Base64: password123

# Use in Pod
spec:
  containers:
  - name: model
    env:
    - name: DB_USERNAME
      valueFrom:
        secretKeyRef:
          name: db-credentials
          key: username
    - name: DB_PASSWORD
      valueFrom:
        secretKeyRef:
          name: db-credentials
          key: password
```

**Best Practices:**
- ✅ **Never commit secrets to Git** (use sealed-secrets, external secrets operator)
- ✅ **Use RBAC:** Restrict who can view secrets
- ✅ **Encrypt at rest:** Enable etcd encryption
- ✅ **Rotate secrets:** Change passwords regularly, update Secrets
- ✅ **ConfigMaps for config:** Secrets only for sensitive data

---

### Section 7: Persistent Storage - Volumes and PersistentVolumeClaims

**Volume Types:**
```yaml
# emptyDir: Temporary, deleted when pod deleted
volumes:
- name: cache
  emptyDir: {}

# hostPath: Mount from node filesystem (not portable)
volumes:
- name: logs
  hostPath:
    path: /var/log/pods

# PersistentVolumeClaim: Durable storage (survives pod deletion)
volumes:
- name: model-data
  persistentVolumeClaim:
    claimName: model-pvc
```

**PersistentVolumeClaim (PVC):**
```yaml
# Request storage
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: model-pvc
spec:
  accessModes:
  - ReadWriteOnce  # RWO (single node), ReadWriteMany (multi-node)
  resources:
    requests:
      storage: 100Gi
  storageClassName: fast-ssd  # AWS gp3, GCP pd-ssd

# Use in StatefulSet
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: redis
spec:
  serviceName: redis
  replicas: 3
  volumeClaimTemplates:  # Automatic PVC per pod
  - metadata:
      name: redis-data
    spec:
      accessModes: ["ReadWriteOnce"]
      resources:
        requests:
          storage: 50Gi
```

**Storage Classes:**
- **AWS:** gp3 (general purpose SSD), io2 (high IOPS), st1 (throughput optimized HDD)
- **GCP:** pd-standard (HDD), pd-ssd (SSD), pd-balanced (balanced SSD)
- **Azure:** Standard_LRS (HDD), Premium_LRS (SSD)

**When to Use:**
- **emptyDir:** Temporary cache, scratch space (deleted on pod restart)
- **PVC:** Model files, database data, logs (persist beyond pod lifecycle)
- **StatefulSet + PVC:** Each pod needs own storage (databases, caches)

---

### Section 8: Namespaces for Multi-Tenancy

**Create Namespace:**
```bash
kubectl create namespace dev
kubectl create namespace staging
kubectl create namespace production
```

**Deploy to Namespace:**
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: wafer-model
  namespace: production  # Deploy to production namespace
```

**Resource Quotas (Prevent Resource Exhaustion):**
```yaml
apiVersion: v1
kind: ResourceQuota
metadata:
  name: production-quota
  namespace: production
spec:
  hard:
    requests.cpu: "100"       # Max 100 CPU cores requested
    requests.memory: "200Gi"  # Max 200GB memory requested
    persistentvolumeclaims: "20"  # Max 20 PVCs
    pods: "100"               # Max 100 pods
```

**Limit Ranges (Default/Max Resources per Pod):**
```yaml
apiVersion: v1
kind: LimitRange
metadata:
  name: production-limits
  namespace: production
spec:
  limits:
  - max:
      cpu: "4"
      memory: "8Gi"
    min:
      cpu: "100m"
      memory: "128Mi"
    default:  # Applied if not specified
      cpu: "500m"
      memory: "512Mi"
    type: Container
```

**Use Cases:**
- **Environment separation:** dev, staging, production (prevent accidental production updates)
- **Team isolation:** team-a, team-b (each team has their own namespace)
- **Cost tracking:** Label namespaces, track spending per team/project

---

### Section 9: Jobs and CronJobs for Batch Processing

**Job (Run to Completion):**
```yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: wafer-analysis
spec:
  parallelism: 10      # 10 pods run in parallel
  completions: 100     # 100 total tasks
  backoffLimit: 3      # Retry up to 3 times on failure
  template:
    spec:
      restartPolicy: OnFailure
      containers:
      - name: analyzer
        image: registry.io/wafer-analyzer:v1.0
        args: ["--process-wafer", "$(WAFER_ID)"]
```

**CronJob (Scheduled Job):**
```yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: nightly-reports
spec:
  schedule: "0 2 * * *"  # 2 AM daily (cron syntax)
  jobTemplate:
    spec:
      template:
        spec:
          restartPolicy: OnFailure
          containers:
          - name: report-generator
            image: registry.io/report-gen:v1.0
```

**Cron Syntax:**
```
# ┌───────────── minute (0 - 59)
# │ ┌───────────── hour (0 - 23)
# │ │ ┌───────────── day of month (1 - 31)
# │ │ │ ┌───────────── month (1 - 12)
# │ │ │ │ ┌───────────── day of week (0 - 6, Sun-Sat)
# │ │ │ │ │
# * * * * *

"0 2 * * *"      # 2 AM daily
"*/15 * * * *"   # Every 15 minutes
"0 0 * * 0"      # Midnight every Sunday
"0 */6 * * *"    # Every 6 hours
```

**When to Use:**
- **Jobs:** One-time batch processing (train model, process dataset)
- **CronJobs:** Scheduled tasks (nightly reports, model retraining)

---

### Section 10: Resource Requests and Limits

**Requests vs Limits:**
```yaml
resources:
  requests:  # Guaranteed resources (scheduler uses for placement)
    cpu: "1"
    memory: "2Gi"
  limits:    # Maximum allowed (enforced by kubelet)
    cpu: "2"
    memory: "4Gi"
```

**What Happens:**
- **CPU request:** Scheduler ensures node has 1 core available before scheduling
- **CPU limit:** Pod throttled if exceeds 2 cores (slowed down, not killed)
- **Memory request:** Scheduler ensures node has 2GB available
- **Memory limit:** Pod killed (OOMKilled) if exceeds 4GB

**QoS Classes (Priority During Resource Contention):**
1. **Guaranteed:** requests = limits (highest priority, killed last)
2. **Burstable:** requests < limits (medium priority)
3. **BestEffort:** No requests/limits (lowest priority, killed first)

**Best Practices:**
- ✅ **Always set requests:** Prevent overcommitment (10 pods × 2GB = 20GB, but node has 16GB)
- ✅ **Set memory limits:** Prevent OOM on node (one pod uses all memory, kills others)
- ✅ **CPU limits optional:** Throttling can reduce performance, often skip limits
- ✅ **Profile your app:** Use Metrics Server to determine actual usage

**View Resource Usage:**
```bash
kubectl top nodes
kubectl top pods -n production
```

---

### Section 11: RBAC (Role-Based Access Control)

**Concepts:**
- **ServiceAccount:** Identity for pods (like IAM role for containers)
- **Role:** Permissions within namespace (can create pods in namespace "dev")
- **ClusterRole:** Permissions cluster-wide (can list nodes)
- **RoleBinding:** Assign Role to user/ServiceAccount
- **ClusterRoleBinding:** Assign ClusterRole cluster-wide

**Example - Read-Only Access to Production:**
```yaml
# Role: View pods in production
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: pod-reader
  namespace: production
rules:
- apiGroups: [""]
  resources: ["pods", "pods/log"]
  verbs: ["get", "list", "watch"]

# RoleBinding: Assign to data-science team
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: read-pods
  namespace: production
subjects:
- kind: Group
  name: data-science-team
  apiGroup: rbac.authorization.k8s.io
roleRef:
  kind: Role
  name: pod-reader
  apiGroup: rbac.authorization.k8s.io
```

**ServiceAccount for Pods:**
```yaml
# ServiceAccount
apiVersion: v1
kind: ServiceAccount
metadata:
  name: model-server-sa
  namespace: production

# Use in Deployment
spec:
  serviceAccountName: model-server-sa
  containers:
  - name: model
    image: registry.io/wafer-model:v2.3
```

**Best Practices:**
- ✅ **Principle of least privilege:** Grant minimum permissions needed
- ✅ **Separate ServiceAccounts:** Different apps get different ServiceAccounts
- ✅ **Avoid default ServiceAccount:** Create specific ServiceAccounts
- ✅ **Audit RBAC:** Regularly review who has access to what

---

### Section 12: Monitoring with Prometheus and Grafana

**Prometheus Architecture:**
```
┌─────────────┐
│   Pods      │ expose /metrics endpoint
│ /metrics    │
└──────┬──────┘
       │
       │ Prometheus scrapes every 15s
       ▼
┌─────────────┐
│ Prometheus  │ stores time-series data
│  Server     │
└──────┬──────┘
       │
       │ PromQL queries
       ▼
┌─────────────┐
│   Grafana   │ visualizes metrics
│ Dashboards  │
└─────────────┘
```

**Expose Metrics:**
```python
from prometheus_client import Counter, Histogram, generate_latest

REQUEST_COUNT = Counter('http_requests_total', 'Total requests')
REQUEST_LATENCY = Histogram('http_request_duration_seconds', 'Request latency')

@app.route('/metrics')
def metrics():
    return generate_latest()
```

**ServiceMonitor (Auto-Discovery):**
```yaml
apiVersion: monitoring.coreos.com/v1
kind: ServiceMonitor
metadata:
  name: wafer-model-metrics
spec:
  selector:
    matchLabels:
      app: wafer-model
  endpoints:
  - port: metrics
    interval: 15s
```

**Key Metrics for ML:**
- **Request rate:** `rate(http_requests_total[5m])`
- **Error rate:** `rate(http_requests_total{status="500"}[5m])`
- **Latency percentiles:** `histogram_quantile(0.99, http_request_duration_seconds)`
- **Model accuracy:** Custom gauge `model_accuracy{model="wafer_yield"}`
- **Prediction distribution:** `prediction_count{label="Pass"}` vs `prediction_count{label="Fail"}`

---

### Section 13: Cost Optimization Strategies

**1. Right-Sizing (Avoid Over-Provisioning):**
```bash
# Check actual usage
kubectl top pods -n production

# If pod requests 2 cores but uses 0.5 cores, reduce request
resources:
  requests:
    cpu: "500m"  # Reduced from 2 cores
    memory: "1Gi"
```

**2. Horizontal Pod Autoscaling:**
- Scale down during low traffic (50 pods → 5 pods, 90% cost savings)
- Scale up during peaks (handle load without over-provisioning)

**3. Cluster Autoscaling:**
- Add nodes when pods pending (insufficient resources)
- Remove nodes when under-utilized (move pods, terminate node)

**4. Spot Instances (70% Discount):**
```yaml
nodeSelector:
  node.kubernetes.io/lifecycle: spot
tolerations:
- key: "spot"
  operator: "Equal"
  value: "true"
  effect: "NoSchedule"
```

**5. Storage Optimization:**
- Delete unused PVCs (orphaned volumes)
- Use lifecycle policies (delete old snapshots)
- Compress logs before storing

**6. Multi-Tenancy (Resource Sharing):**
- Run dev/staging/production on same cluster (separate namespaces)
- Share cluster overhead (control plane, monitoring) across teams

**Cost Comparison:**
| Strategy | Monthly Cost | Savings |
|----------|--------------|---------|
| Static 50 pods (m5.2xlarge) | $5,000 | - |
| HPA (avg 15 pods) | $1,500 | 70% |
| HPA + Spot instances | $450 | 91% |
| HPA + Spot + Right-sizing | $300 | 94% |

---

### Section 14: Troubleshooting Common Issues

**Issue 1: Pod Stuck in Pending**
```bash
# Check pod status
kubectl describe pod <pod-name>

# Common causes:
# - Insufficient resources (CPU/memory)
# - Node selector mismatch (no nodes match label)
# - PVC not bound (storage class issue)
# - Image pull error (authentication)

# Solutions:
# - Add nodes or reduce resource requests
# - Fix node labels or remove node selector
# - Check storage class, PVC status
# - Verify image pull secret
```

**Issue 2: Pod CrashLoopBackOff**
```bash
# View logs
kubectl logs <pod-name>
kubectl logs <pod-name> --previous  # Previous crashed container

# Common causes:
# - Application error (bugs, missing config)
# - Liveness probe failing
# - OOMKilled (exceeds memory limit)

# Solutions:
# - Fix application code
# - Adjust probe thresholds
# - Increase memory limit
```

**Issue 3: Service Not Reachable**
```bash
# Check service endpoints
kubectl get endpoints <service-name>

# If no endpoints:
# - Selector mismatch (Service selector doesn't match Pod labels)
# - Pods not ready (readiness probe failing)

# Test from another pod
kubectl run -it --rm debug --image=busybox --restart=Never -- wget -O- http://service-name:8080
```

**Issue 4: High Latency**
```bash
# Check pod resource usage
kubectl top pods

# If CPU throttled:
# - Increase CPU limit or remove limit
# - Scale out (more replicas)

# If network latency:
# - Enable network policies (restrict traffic)
# - Use Service mesh (Istio) for advanced routing
```

---

### Section 15: Production Deployment Checklist

**Before Going to Production:**

✅ **Reliability:**
- [ ] Min 3 replicas (survive pod failures)
- [ ] PodDisruptionBudget configured (min 2 available during node drain)
- [ ] Liveness and readiness probes configured
- [ ] Resource requests and limits set
- [ ] Anti-affinity rules (spread pods across zones)

✅ **Scalability:**
- [ ] HPA configured (auto-scale based on CPU/custom metrics)
- [ ] Cluster autoscaler enabled (auto-scale nodes)
- [ ] Load testing performed (verify handles expected traffic)

✅ **Security:**
- [ ] RBAC configured (least privilege)
- [ ] Network policies (restrict pod-to-pod traffic)
- [ ] Secrets for credentials (not environment variables)
- [ ] Container images scanned (Trivy, Snyk)
- [ ] Non-root user in containers

✅ **Observability:**
- [ ] Prometheus metrics exposed
- [ ] Grafana dashboards created
- [ ] Alerts configured (error rate, latency, resource usage)
- [ ] Distributed tracing (Jaeger, Zipkin)
- [ ] Log aggregation (ELK, Loki)

✅ **Deployment:**
- [ ] Rolling update strategy (zero downtime)
- [ ] Rollback tested (verify can revert quickly)
- [ ] Blue-green or canary deployment (for critical services)
- [ ] Staging environment (test before production)

✅ **Disaster Recovery:**
- [ ] Backups configured (etcd, persistent volumes)
- [ ] Multi-AZ deployment (survive zone failures)
- [ ] Runbook documented (incident response procedures)
- [ ] Chaos engineering tested (simulate failures)

---

### Section 16: Kubernetes vs Alternatives

**When to Use Kubernetes:**
- ✅ Microservices (100+ services, need orchestration)
- ✅ Multi-cloud (same YAML runs on AWS, GCP, Azure)
- ✅ Auto-scaling (handle variable load)
- ✅ High availability (99.99% uptime requirement)

**When to Consider Alternatives:**
- **AWS ECS:** Simpler, AWS-only (easier for small teams, less portable)
- **Docker Compose:** Local development (single machine, not production)
- **Serverless (Lambda, Cloud Functions):** Event-driven, auto-scaling (no cluster management)
- **Nomad (HashiCorp):** Lighter than Kubernetes (easier to operate, less ecosystem)

**Kubernetes Learning Curve:**
- Week 1: Pods, Deployments, Services (basic deployments)
- Week 2-4: HPA, ConfigMaps, Secrets, Volumes (production features)
- Month 2-3: StatefulSets, Ingress, RBAC, Operators (advanced patterns)
- Month 4-6: Multi-cluster, Service mesh, GitOps (expert level)

---

### Section 17: Next Steps - Kubernetes Advanced Patterns

**What We've Learned (Fundamentals):**
- ✅ Kubernetes architecture (control plane, nodes, pods)
- ✅ Deployments and rolling updates (zero-downtime deployments)
- ✅ Services and networking (load balancing, service discovery)
- ✅ Auto-scaling (HPA for cost optimization)
- ✅ ConfigMaps, Secrets (externalize configuration)

**What's Next (Advanced - Notebook 133):**
- **StatefulSets:** Databases, caches with stable network IDs
- **DaemonSets:** Run pod on every node (monitoring agents, log collectors)
- **Operators:** Automate complex applications (Postgres Operator, Redis Operator)
- **Custom Resource Definitions (CRDs):** Extend Kubernetes API
- **Init Containers:** Pre-flight checks before main container starts
- **Admission Controllers:** Policy enforcement (mutate/validate resources)
- **Network Policies:** Firewall rules for pods (restrict traffic)
- **Pod Security Policies:** Security standards enforcement

**The Journey Continues:**
```
Notebook 132: Kubernetes Fundamentals ✅
  ↓
Notebook 133: Kubernetes Advanced (StatefulSets, Operators, CRDs)
  ↓
Notebook 134: Service Mesh with Istio (traffic management, mTLS, observability)
  ↓
Notebook 135: GitOps & ArgoCD (declarative, automated deployments)
  ↓
Notebook 136: CI/CD for Kubernetes (Jenkins, GitHub Actions, Tekton)
  ↓
Notebook 137: Multi-Cloud Kubernetes (EKS, GKE, AKS comparison)
  ↓
Notebook 138: Production ML on Kubernetes (Kubeflow, KServe, Seldon)
```

---

### Section 18: Quick Reference - Essential kubectl Commands

**Cluster Info:**
```bash
kubectl cluster-info
kubectl get nodes
kubectl top nodes
kubectl describe node <node-name>
```

**Pod Management:**
```bash
kubectl get pods -n production
kubectl describe pod <pod-name>
kubectl logs <pod-name> -f
kubectl logs <pod-name> -c <container-name>
kubectl exec -it <pod-name> -- bash
kubectl delete pod <pod-name>
kubectl port-forward <pod-name> 8080:8080
```

**Deployment Management:**
```bash
kubectl create deployment wafer-model --image=registry.io/wafer-model:v2.3
kubectl get deployments
kubectl describe deployment wafer-model
kubectl scale deployment wafer-model --replicas=10
kubectl set image deployment/wafer-model model=registry.io/wafer-model:v2.4
kubectl rollout status deployment/wafer-model
kubectl rollout history deployment/wafer-model
kubectl rollout undo deployment/wafer-model
kubectl delete deployment wafer-model
```

**Service Management:**
```bash
kubectl expose deployment wafer-model --port=8080 --type=LoadBalancer
kubectl get services
kubectl describe service wafer-model
kubectl delete service wafer-model
```

**ConfigMap & Secret:**
```bash
kubectl create configmap model-config --from-file=config.yaml
kubectl create secret generic db-creds --from-literal=password=secret123
kubectl get configmaps
kubectl get secrets
kubectl describe configmap model-config
```

**Namespace:**
```bash
kubectl create namespace production
kubectl get namespaces
kubectl config set-context --current --namespace=production
kubectl delete namespace dev
```

**Apply YAML:**
```bash
kubectl apply -f deployment.yaml
kubectl apply -f . (all YAML in directory)
kubectl delete -f deployment.yaml
```

**Debugging:**
```bash
kubectl describe pod <pod-name>  # Events, status
kubectl logs <pod-name> --previous  # Logs from crashed container
kubectl top pods  # Resource usage
kubectl get events --sort-by='.lastTimestamp'
kubectl run -it --rm debug --image=busybox --restart=Never -- sh
```

---

**🎉 You've Mastered Kubernetes Fundamentals!**

**Key Achievements:**
- ✅ Deployed ML models on Kubernetes (10 replicas, auto-scaling)
- ✅ Implemented zero-downtime updates (rolling updates, rollbacks)
- ✅ Configured auto-scaling (HPA, 70% cost savings)
- ✅ Mastered Kubernetes architecture (control plane, nodes, pods, services)
- ✅ Built production-ready deployments (health checks, resource limits, monitoring)

**Real-World Impact:**
- **Scalability:** 5 → 500 pods automatically (handle 100x traffic spike)
- **Availability:** 99.99% uptime (4 minutes downtime/year)
- **Cost:** 70-90% savings with HPA + spot instances
- **Speed:** 5-minute deployments (vs 8-hour manual deployments)
- **Reliability:** Self-healing (crashed pods auto-restart in 10 seconds)

**Keep Learning:** Notebook 133 awaits - Advanced Kubernetes patterns for production ML! 🚀

## 🎯 Key Takeaways

### When to Use Kubernetes for ML
- **Multi-model serving**: Deploy 10+ models with different resource needs (CPU-bound vs. GPU-bound)
- **Autoscaling**: Handle variable inference traffic (10 req/s → 1000 req/s spikes)
- **Resource efficiency**: Share GPU cluster across training jobs, maximize utilization >80%
- **Team collaboration**: Multiple data scientists sharing infrastructure (namespace isolation)
- **CI/CD integration**: Automated model deployment pipelines (Jenkins, GitLab CI → K8s)

### Limitations
- **Complexity overhead**: Learning curve steep (pods, services, ingress, persistent volumes)
- **Operational burden**: Managing K8s cluster requires DevOps expertise (upgrades, security patches)
- **Cost**: Managed K8s (EKS, GKE, AKS) adds $70-150/month per cluster + node costs
- **Overkill for simple use cases**: Single model serving doesn't need full orchestration
- **GPU scheduling challenges**: Sharing GPUs across pods complex (requires device plugins)

### Alternatives
- **Docker Compose**: Simpler multi-container orchestration for development/small deployments
- **Serverless (AWS Lambda, Cloud Run)**: For lightweight models, pay-per-inference, no cluster management
- **Dedicated VM**: Single model on VM with systemd service (simple, predictable)
- **Managed ML platforms**: SageMaker, Vertex AI handle infrastructure (less control, easier operation)

### Best Practices
- **Resource requests/limits**: Set CPU/memory requests for scheduling, limits to prevent OOM kills
- **Health checks**: Liveness (restart if unhealthy) + readiness (traffic only when ready)
- **Horizontal pod autoscaling**: Scale replicas based on CPU/memory/custom metrics (e.g., queue depth)
- **Namespace isolation**: Separate dev/staging/prod environments
- **ConfigMaps/Secrets**: Externalize configuration (model paths, API keys) from container images
- **Persistent volumes**: Use for model artifacts, training data (StatefulSets for databases)

## 🔍 Diagnostic Checks & Mastery

### Implementation Checklist
- ✅ **K8s cluster**: Minikube (local) or managed (EKS, GKE, AKS)
- ✅ **ML deployment**: Deploy model as Deployment + Service (replicas for HA)
- ✅ **Resource management**: Set requests/limits for CPU, memory, GPU
- ✅ **Autoscaling**: Configure HPA based on CPU/memory metrics
- ✅ **Storage**: PersistentVolumeClaims for model artifacts, training data
- ✅ **ConfigMaps/Secrets**: Externalize config (API keys, model paths)

### Post-Silicon Applications
**ATE Test Server Deployment**: Deploy yield prediction models across 10 ATE test cells, autoscale based on test queue depth, save $1.5M/year ATE utilization

### Mastery Achievement
✅ Deploy ML models to Kubernetes with replicas and load balancing  
✅ Configure autoscaling (HPA) for variable inference traffic  
✅ Manage GPU resources efficiently across training/serving workloads  
✅ Use ConfigMaps/Secrets for configuration management  
✅ Apply to semiconductor test infrastructure deployment  

**Next Steps**: 133_Kubernetes_Advanced_Patterns, 134_Service_Mesh_Istio_Linkerd

## 📈 Progress Update

**Session Summary:**
- ✅ Completed 29 notebooks total (previous 21 + current batch: 132, 134-136, 139, 144-145, 174)
- ✅ Current notebook: 132/175 complete
- ✅ Overall completion: ~82.9% (145/175 notebooks ≥15 cells)

**Remaining Work:**
- 🔄 Next: Process remaining 9-cell and below notebooks
- 🎯 Target: 100% completion (175/175 notebooks)

Excellent progress - over 80% complete! 🚀

In [ ]:
# gpu-training-job.yaml
"""
apiVersion: batch/v1
kind: Job
metadata:
  name: wafer-cnn-training
spec:
  template:
    spec:
      containers:
      - name: pytorch-training
        image: pytorch/pytorch:2.0-cuda11.7
        command: ["python", "/app/train_wafer_cnn.py"]
        resources:
          limits:
            nvidia.com/gpu: 2  # Request 2 GPUs
          requests:
            memory: "16Gi"
            cpu: "4"
        volumeMounts:
        - name: training-data
          mountPath: /data
        - name: model-output
          mountPath: /models
      nodeSelector:
        gpu: "true"  # Schedule on GPU nodes only
      volumes:
      - name: training-data
        persistentVolumeClaim:
          claimName: wafer-data-pvc
      - name: model-output
        persistentVolumeClaim:
          claimName: model-storage-pvc
      restartPolicy: Never
  backoffLimit: 3  # Retry up to 3 times on failure
"""

# Apply with:
# kubectl apply -f gpu-training-job.yaml

# Monitor job status
# kubectl get jobs
# kubectl logs job/wafer-cnn-training

# Post-Silicon Use Case:
# Train CNN on 50K wafer map images for defect classification
# 2 GPUs reduce training from 8 hours (CPU) to 45 minutes
# Deploy weekly training jobs triggered by new ATE data
# Save $120K/year (4 hours SRE time/week × $150K salary)

## 🏭 Advanced Example: GPU-Enabled ML Training Job

Deploy PyTorch distributed training on Kubernetes with GPU node affinity.